In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset
from lib.ehr.icu_interface import Inpatients

In [3]:
import logging
logging.root.level = logging.DEBUG


In [4]:
# Assign the folder of the dataset to `DATA_FILE`.

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")

with U.modified_environ(DATA_DIR=DATA_DIR):
    m4inpatient_dataset = load_dataset('M4ICU', max_workers=1)
   

DEBUG:root:Loading dataframe files
DEBUG:root:[DONE] Loading dataframe files
DEBUG:root:Matching admission_id
DEBUG:root:[DONE] Matching admission_id
DEBUG:root:Time casting..
DEBUG:root:[DONE] Time casting..


INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


DEBUG:root:Dataframes validation and time conversion
INFO:root:Unrecognised ICD v10 codes: 3323 (28.74%)
DEBUG:root:
                    Unrecognised <class 'lib.ehr.coding_scheme.DxICD10'> codes (3323)
                    to be removed: ['E08.3513', 'E10.3213', 'E10.3219', 'E10.3291', 'E10.3293', 'E10.3299', 'E10.3313', 'E10.3319', 'E10.3393', 'E10.3399', 'E10.3411', 'E10.3413', 'E10.3491', 'E10.3511', 'E10.3512', 'E10.3513', 'E10.3519', 'E10.3522', 'E10.3523', 'E10.3531', 'E10.3532', 'E10.3559', 'E10.3591', 'E10.3592', 'E10.3593', 'E10.3599', 'E11.3213', 'E11.3219', 'E11.3291', 'E11.3292', 'E11.3293', 'E11.3299', 'E11.3311', 'E11.3313', 'E11.3319', 'E11.3391', 'E11.3393', 'E11.3399', 'E11.3413', 'E11.3419', 'E11.3491', 'E11.3492', 'E11.3493', 'E11.3499', 'E11.3513', 'E11.3519', 'E11.3521', 'E11.3532', 'E11.3542', 'E11.3553', 'E11.3591', 'E11.3592', 'E11.3593', 'E11.3599', 'H34.8112', 'H34.8120', 'H34.8122', 'H34.8192', 'H34.8310', 'H34.8320', 'H35.3110', 'H35.3120', 'H35.3130', 'H35.

INFO:root:Unrecognised ICD v9 codes: 118 (1.63%)
DEBUG:root:
                    Unrecognised <class 'lib.ehr.coding_scheme.DxICD9'> codes (118)
                    to be removed: ['041.49', '173.21', '173.22', '173.30', '173.31', '173.32', '173.40', '173.41', '173.42', '173.50', '173.51', '173.52', '173.59', '173.60', '173.61', '173.62', '173.70', '173.71', '173.72', '173.79', '173.80', '173.81', '173.82', '173.91', '173.92', '173.99', '282.40', '282.43', '282.44', '282.46', '284.11', '284.12', '284.19', '286.52', '286.53', '286.59', '294.20', '294.21', '310.81', '310.89', '331.6', '348.82', '358.30', '365.70', '365.72', '365.73', '414.4', '415.13', '425.11', '425.18', '444.09', '488.81', '488.82', '488.89', '512.2', '512.82', '512.83', '512.84', '512.89', '516.31', '516.32', '516.33', '516.34', '516.35', '516.36', '516.37', '516.4', '516.5', '518.51', '518.52', '518.53', '539.01', '539.09', '539.81', '539.89', '573.5', '596.81', '596.82', '596.83', '596.89', '629.31', '649.81', '704.

In [5]:
splits = m4inpatient_dataset.random_splits(0.8, 0.9, 42)

In [6]:
preprocessing = m4inpatient_dataset.fit_preprocessing(splits[0])

In [7]:
m4inpatient_dataset.apply_preprocessing(preprocessing)

DEBUG:root:Removed 2320851 (0.023) outliers from obs


In [26]:
m4inaptients = Inpatients(m4inpatient_dataset, splits[2][:10])

DEBUG:root:Loading subjects..


INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


DEBUG:root:Extracting dx codes...
DEBUG:root:[DONE] Extracting dx codes
DEBUG:root:Extracting dx codes history...
DEBUG:root:[DONE] Extracting dx codes history
DEBUG:root:Extracting outcome...
DEBUG:root:[DONE] Extracting outcome
DEBUG:root:Extracting procedures...
DEBUG:root:[DONE] Extracting procedures
DEBUG:root:Extracting inputs...
DEBUG:root:[DONE] Extracting inputs
DEBUG:root:Extracting observables...
DEBUG:root:[DONE] Extracting observables
DEBUG:root:[DONE] Loading subjects


In [28]:
m4inaptients.size_in_bytes / 1024 ** 3

0.0014968374744057655

In [20]:
m4inaptients_jax = m4inaptients.to_jax_arrays(splits[2])

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00032019615173339844 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140049084490352) for with global shapes and types (ShapedArray(float32[67]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0001862049102783203 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140049084490352) for with global shapes and types (ShapedArray(float16[67,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00020456314086914062 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140049084493632) for with global shapes and types (ShapedArray(bool[67,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00019669532775878906 sec
DEBU

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0005047321319580078 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140052425308896) for with global shapes and types (ShapedArray(float16[90,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002143383026123047 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140052425308736) for with global shapes and types (ShapedArray(bool[90,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0003857612609863281 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140052425306336) for with global shapes and types (ShapedArray(int32[70]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00038313865661621094 sec
DEBUG:ja

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00022935867309570312 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140047018710576) for with global shapes and types (ShapedArray(float16[68,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0006961822509765625 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140047018711616) for with global shapes and types (ShapedArray(bool[68,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002810955047607422 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140047018709696) for with global shapes and types (ShapedArray(int32[55]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00020551681518554688 sec
DEBUG:j

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00047898292541503906 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140053660157248) for with global shapes and types (ShapedArray(float32[131]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00027060508728027344 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140053660159648) for with global shapes and types (ShapedArray(float32[53]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00020241737365722656 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140053660159328) for with global shapes and types (ShapedArray(float16[53,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00041484832763671875 sec
DE

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002033710479736328 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140052485327104) for with global shapes and types (ShapedArray(float16[54,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00021266937255859375 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140052485328224) for with global shapes and types (ShapedArray(bool[54,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002651214599609375 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054269374192) for with global shapes and types (ShapedArray(float16[11,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00019431114196777344 sec
DE

...

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00019025802612304688 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054298954304) for with global shapes and types (ShapedArray(int32[1059]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002601146697998047 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054298954624) for with global shapes and types (ShapedArray(float32[1059]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002846717834472656 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054298691280) for with global shapes and types (ShapedArray(float16[249,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00019431114196777344 sec
DE

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00018477439880371094 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054292187792) for with global shapes and types (ShapedArray(float16[1155,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00021409988403320312 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054292188352) for with global shapes and types (ShapedArray(bool[1155,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0001862049102783203 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054292188912) for with global shapes and types (ShapedArray(int32[1890]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00018835067749023438 sec


DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0001914501190185547 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054284417040) for with global shapes and types (ShapedArray(float16[883,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00018906593322753906 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054284418960) for with global shapes and types (ShapedArray(bool[883,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00018668174743652344 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054284417920) for with global shapes and types (ShapedArray(int32[856]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0001862049102783203 sec
DEBU

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0001862049102783203 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054274521024) for with global shapes and types (ShapedArray(float32[1014]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002770423889160156 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054273463856) for with global shapes and types (ShapedArray(int32[1519]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00022101402282714844 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054273464256) for with global shapes and types (ShapedArray(float32[1519]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00025081634521484375 sec
DEBU

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002765655517578125 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054263810784) for with global shapes and types (ShapedArray(int32[552]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.000244140625 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054258666608) for with global shapes and types (ShapedArray(float32[619]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00023651123046875 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140054258667008) for with global shapes and types (ShapedArray(float16[619,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00021338462829589844 sec
DEBUG:jax.inte

In [21]:
m4inaptients_jax.size_in_bytes / 1024 ** 3

0.7668210975825787

In [23]:
m4inaptients_jax.subjects[splits[2][0]]

Inpatient(
  subject_id='18326767',
  static_info=StaticInfo(
    gender='F',
    date_of_birth=Timestamp('2104-01-01 00:00:00'),
    ethnicity=bool[5],
    ethnicity_scheme=<lib.ehr.coding_scheme.MIMIC4Eth5 object at 0x7f60b53dc670>,
    constant_vec=bool[6]
  ),
  admissions=[
    InpatientAdmission(
      admission_id='26809767',
      admission_dates=(
        Timestamp('2153-08-01 21:09:00'),
        Timestamp('2153-08-03 16:18:00')
      ),
      dx_codes=CodesVector(
        vec=bool[17375],
        scheme=<lib.ehr.coding_scheme.DxICD9 object at 0x7f5f11164400>
      ),
      dx_codes_history=CodesVector(
        vec=bool[17375],
        scheme=<lib.ehr.coding_scheme.DxICD9 object at 0x7f5f11164400>
      ),
      outcome=CodesVector(
        vec=bool[2081],
        scheme=<lib.ehr.outcome.OutcomeExtractor object at 0x7f60b53dc0d0>
      ),
      observables=InpatientObservables(
        time=f32[3],
        value=f16[3,60],
        mask=bool[3,60]
      ),
      interventions=I

## TODO

1. Squeeze code vectors.
2. Squeeze float32 to float16.